# Summary

```
by Seokkyu Kong
Date: 2016-05-02
```

참고 자료: 1 [모두를 위한 머신러닝/딥러닝 강의 - 홍콩과기대 김성훈교수님](http://hunkim.github.io/ml/)

참고 자료: 2, ML lab 04 - multi-variable linear regression을 TensorFlow에서 구현하기  
https://www.youtube.com/watch?v=iEaVR1N8EEk&feature=youtu.be

위의 강좌 랩에서 다변량 선형 회귀에 대한 tensorflow 활용예제가 나온다.
위의 예제를 활용해서 코세라 머신러닝 강좌 1의 다변량 선형 회귀 연습문제에 적용해서 결과값을 서로 비교해 본다.

한 가지 주의할 점은 김성훈교수님의 강의 랩에서는 training set 의 입력이 nxm 형태이며 bias term 이 파일 자료에 포함되어 있다. 여기서 m은 training 데이터의 갯수이며, n은 feature 갯수이다. 이런 데이터 처리는 np.loadtxt('xxx', unpack=True, dtype='flat32') 와 같이 unpack=True 지정으로 가능하다.

위와 같은 nxm 차원의 데이터는 코세라 연습문제와 같은 mxn 포맷이 아니라서 헷갈린다.

따라서 기존 예제와 달리 아래 내용을 변경해본다.

- training dataset 은 mxn 으로 읽어서 처리한다. 
- bias term을 포함시킨다.
- 정규화를 적용한다. featureNormalize()는 코세라 연습문제에서 빌어온다.
- 새로운 test dataset을 예측하기 위해 placeholder를 사용한다.

In [3]:
%pylab inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [9]:

def featureNormalize(X):
    """
    피쳐 정규화 함수: 각각의 피쳐는 단위가 다르기 때문에 평균이 0이고 표준편차가 1인
    정규화 값으로 변환된다.
    """
    X_norm = X
    n = np.size(X, 1)
    mu = np.zeros((1, n))
    sigma = np.zeros((1, n))
    
    # 0은 column 단위 연산, 1은 row 단위 연산 적용
    m = np.size(X, 0)
    mu = mean(X_norm, 0)
    sigma = std(X_norm, 0)
    
    X_norm = (X_norm - mu) / sigma
    
    return X_norm, mu, sigma

"""
# 기존 김성훈교수님 강의 랩에서 데이터를 읽어 들이는 방식

xy = np.loadtxt('train.txt', unpack=True, dtype='float32')

x_data = xy[0:-1]
y_data = xy[-1]
"""

# mxn 으로 데이터를 읽어들어서 처리한다.
xy = np.loadtxt('ex1data2.txt', dtype='float32', delimiter = ',')
x_data = xy[:, 0:-1]
y_data = xy[:, -1]

# 정규화를 수행한다.
X_norm, mu, sigma = featureNormalize(x_data)

m = x_data.shape[0] # 자료의 갯수

x_data = np.column_stack((np.ones(m), X_norm)) # bias column vector 추가
y_data = y_data.reshape((m, 1)) # y_data dimension을 mx1로 맞추어 준다.

m, n = x_data.shape

# X,Y 에 대한 placeholder 선언
X = tf.placeholder(tf.float32, [None, n]) # X: mxn dimension
Y = tf.placeholder(tf.float32, [None, 1]) # Y: mx1 dimension

# 파라미터 초기화 (구하고자 하는 대상 값임)
W = tf.Variable(tf.random_uniform([n, 1], -1.0, 1.0)) # nx1, OK
#W = tf.Variable(tf.zeros([3, 1])) # OK

# 가설함수 정의, X, W의 순서를 생각해라.
hypothesis = tf.matmul(X, W)

# cost function 정의
cost = tf.reduce_mean(tf.square(hypothesis - Y))

print "X: ", tf.Print(X, [X])
print "Y: ", tf.Print(Y, [Y])
print "W: ", tf.Print(W, [W])
print "hypothesis: ", tf.Print(hypothesis, [hypothesis])
print "cost: ", tf.Print(cost, [cost])
print 

# 제곱합 최소화 하기
# learning rate alpha 값을 다르게 하면서 cost J의 변화 정도를 
# 그래프로 확인해본다.
a = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# 변수 초기화
init = tf.initialize_all_variables()

# Graph 시작
sess = tf.Session()
sess.run(init)


# cost function이 최소가 되는 파라미터 W를 찾는다.
for step in xrange(101):
    sess.run(train, feed_dict = {X: x_data, Y: y_data})
    if step % 20 == 0:
        print step, sess.run(cost, feed_dict = {X: x_data, Y: y_data})  \
            , sess.run(W, feed_dict = {X: x_data, Y: y_data})
    
# 1650 평방피트, 방 3개의 가격을 예측한다.
x_test = np.array([1650, 3])
# 정규화 시킨다.
x_test = (x_test - mu) / sigma

x_test = np.append(1, x_test)
x_test = x_test.reshape((1, 3))

print "Predicted price of a 1650 sq-ft, 3 br house (using gradient descent): "
print "price = ", sess.run(hypothesis, feed_dict = {X: x_test})

"""
In Octave/MATLAB: 

best theta for gradient descent... 
theta =

   3.3866e+05
   1.0413e+05
  -1.7221e+02

Predicted price of a 1650 sq-ft, 3 br house (using gradient descent):
 $292748.085232
"""
print

X:  Tensor("Print_26:0", shape=(?, 3), dtype=float32)
Y:  Tensor("Print_27:0", shape=(?, 1), dtype=float32)
W:  Tensor("Print_28:0", shape=(3, 1), dtype=float32)
hypothesis:  Tensor("Print_29:0", shape=(?, 1), dtype=float32)
cost:  Tensor("Print_30:0", shape=(), dtype=float32)

0 8.46207e+10 [[ 68082.7890625 ]
 [ 21152.26953125]
 [ 10942.15039062]]
20 4.15825e+09 [[ 337272.90625   ]
 [ 101045.640625  ]
 [   1783.82861328]]
40 4.08811e+09 [[ 340376.46875  ]
 [ 108119.8046875]
 [  -5250.390625 ]]
60 4.0866e+09 [[ 340412.21875   ]
 [ 109237.40625   ]
 [  -6367.96630859]]
80 4.08656e+09 [[ 340412.625     ]
 [ 109414.453125  ]
 [  -6545.02148438]]
100 4.08656e+09 [[ 340412.625     ]
 [ 109442.5078125 ]
 [  -6573.07177734]]
Predicted price of a 1650 sq-ft, 3 br house (using gradient descent): 
price =  [[ 293082.5625]]

